# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [20]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [21]:
ws = Workspace.from_config()
experiment_name = 'Capstone_hyp4'

experiment=Experiment(ws, experiment_name)

In [22]:
#create comute cluster
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "proj3cluster"


# Verify that the cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [24]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({'--C': uniform(0.1, 0.4), '--max_iter': choice(12,25,50)})

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

#TODO: Create your estimator and hyperdrive config
estimator = ScriptRunConfig(source_directory=".",
                      script='train.py',
                      compute_target=cpu_cluster,
                      environment=sklearn_env)

hyperdrive_run_config = HyperDriveConfig(run_config=estimator,
                              hyperparameter_sampling=param_sampling,
                              policy=early_termination_policy,
                              primary_metric_name="accuracy",
                              primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                              max_total_runs=20,
                              max_concurrent_runs=4)

In [25]:
#TODO: Submit your experiment

hyperdrive_run = experiment.submit(config=hyperdrive_run_config)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [26]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()
#hyperdrive_run.wait_for_completion(show_output=False)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

KeyError: 'log_files'

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [33]:
import joblib

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_metrics=best_run.get_metrics()
print(f"Best Run ID: {best_run.id}")
print(f"Best accuracy: {best_metrics['accuracy']}")
    


KeyboardInterrupt: 

In [22]:
#TODO: Save the best model
final_model_hyp = best_run.register_model(model_name = 'hyp_best_run',model_path = 'outputs/model.joblib')


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [32]:

from azureml.core.model import Model
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script = 'score.py', environment = sklearn_env)

deployment_config = AciWebservice.deploy_configuration( auth_enabled= True, 
                                                    enable_app_insights= True)

service = Model.deploy(ws, "hyp-best-model", [final_model_hyp], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'


KeyboardInterrupt: 

TODO: In the cell below, send a request to the web service you deployed to test it.

In [28]:
from azureml.core import Webservice
import json


# Specify the deployed service name
service_name = 'hyp-best-model'

# Get the deployed web service
service = Webservice(ws, name=service_name)


input_data = {
    'data': [
        {'Pregnancies': 6, 'Glucose': 8, 'BloodPressure':72, 'SkinThickness':35, 'Insulin':0,
    'BMI':33.6,'DiabetesPedigreeFunction':0.627,'Age':50}  
    ]
}
# Convert input data to JSON format
input_data_json = json.dumps(input_data)

# Send a request to the web service
output = service.run(input_data_json)

# Print the predicted output
print(output)


KeyboardInterrupt: 

TODO: In the cell below, print the logs of the web service and delete the service

In [29]:
print(service.get_logs())


KeyboardInterrupt: 

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

